In [5]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor
from IPython.display import Audio as IPAudio

In [2]:
# dummy dataset, however you can swap this with an dataset on the 🤗 hub or bring your own
librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

In [3]:
# load the model + processor (for pre-processing the audio)
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

/home/btech/nityanand.mathur/miniconda3/envs/bark/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [4]:
# cast the audio data to the correct sampling rate for the model
librispeech_dummy = librispeech_dummy.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
audio_sample = librispeech_dummy[0]["audio"]["array"]

In [35]:
print(audio_sample.shape)

(140520,)


In [6]:
IPAudio(audio_sample, rate=processor.sampling_rate)

In [7]:
# pre-process the inputs
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

In [17]:
print('Sampling Rate =', processor.sampling_rate)
print('Shape of i/p to Encodec model', inputs.input_values.shape)

Sampling Rate = 24000
Shape of i/p to Encodec model torch.Size([1, 1, 140520])


In [19]:
# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])

In [23]:
print('Audio Codes Shape =', encoder_outputs.audio_codes.shape)
print('Audio Codes =', encoder_outputs.audio_codes)

Audio Codes Shape = torch.Size([1, 1, 2, 440])
Audio Codes = tensor([[[[  62,  835,  835,  835,  835,  835,  835,  835,  408,  408,  835,
            835,  835,  835,  835,  408,  408,  408,  408,  408,  408,  408,
            408,  408,  408,  408,  408,  408,  408,  408,  408,  408,  408,
            408,  408,  408,  408,  408,  408,  408,  339,  228,  570,  991,
            681,  972,  969,  303,   38,  463,  738,  106,  855,  602,  142,
            511,  722,  860,  604,  876,  738,  106, 1014,  405,  405,  488,
            461,  461,  461,  293,  736,  933,  894,  723,  784,  837,  291,
           1000,   52, 1019,  488,  854,  872,  585,  991,  784,  723,   11,
            722,  722,  722,  681,  723,  723,  723,  681,  734,  825,  534,
            972,  303,   53,   53,  463,  463,   53,  373,  373,  136,  991,
            840,  765,  407,  303,  840,  916,  613,  393,  430,  876,  738,
            865,  408,  738,  904,   40,  103,  731,  731,  935,  935,   80,
             20

In [24]:
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]

In [26]:
print('Shape of o/p from decoder of encodec', audio_values.shape)

Shape of o/p from decoder of encodec torch.Size([1, 1, 140520])


In [29]:
IPAudio(audio_values[0].detach().cpu().numpy(), rate=processor.sampling_rate)